In [20]:
import os
import gc
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score
import tensorflow as tf
# 001
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121, InceptionV3, EfficientNetB0, Xception, MobileNetV2
from tensorflow.keras import models, layers, optimizers, callbacks
from tensorflow.keras import backend as K

# 002
# from keras._tf_keras.keras.preprocessing.image import  ImageDataGenerator
# from keras._tf_keras.keras.applications import VGG16, ResNet50, DenseNet121, InceptionV3, EfficientNetB0, Xception, MobileNetV2
# from keras._tf_keras.keras import models, layers, optimizers, callbacks
# from keras._tf_keras.keras import backend as K


print("Todas as bibliotecas foram importadas com sucesso!")

Todas as bibliotecas foram importadas com sucesso!


In [21]:
# Criando função para calcular as métricas

def get_metrics(y_true, y_pred):
    vn, fp, fn, vp = confusion_matrix(y_true, y_pred).ravel()
    accuracy = (vp + vn) / (vp + fp + fn + vn)  
    recall = vp / (vp + fn)
    specificity = vn / (vn + fp)
    precision = vp / (vp + fp)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pred)

    return {
        'accuracy': accuracy,
        'specificity': specificity,
        'recall': recall,
        'precision': precision,
        'f1-score': f1,
        'kappa': kappa,
        'auc-roc': auc_roc
    }


In [22]:
# Criando função para seleção de esquema de cor 
def convert_color_scale(image, scale):
    if scale == 'hsv':
        return cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    elif scale == 'rgb':
        return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    elif scale == 'grayscale':
        # Converter para escala de cinza e replicar para 3 canais
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        return cv2.merge([gray, gray, gray])
    elif scale == 'lab':
        return cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    elif scale == 'luv':
        return cv2.cvtColor(image, cv2.COLOR_BGR2LUV)
    elif scale == 'xyz':
        return cv2.cvtColor(image, cv2.COLOR_BGR2XYZ)
    else:
        raise ValueError("Escala de cor não suportada.")

In [23]:
# Carregamento e pré-processamento de imagens com escolha de escala de cor

def load_images(folder, color_scale, img_extensions):
    images = []
    for filename in os.listdir(folder):
        if any(filename.lower().endswith(ext) for ext in img_extensions):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (224, 224))  # Ajuste o tamanho conforme necessário
                img = convert_color_scale(img, color_scale)  # Converta para a escala de cor desejada
                # Se a imagem estiver em escala de cinza, expanda as dimensões
                if color_scale == 'grayscale':
                    img = np.expand_dims(img, axis=-1)  # Adiciona uma dimensão de canal
                images.append(img)

    return np.array(images)


In [24]:
# Defina as pastas de dados
data_dir = r"C:\Users\andre\Pictures\OvarianCancer"
normal_dir = os.path.join(data_dir, 'Non_Cancerous')
cancer_dir = os.path.join(data_dir, 'Endometri')
img_extensions = ['.jpg', '.jpeg', '.png']

if os.path.exists(normal_dir): print(f"Pasta normal_dir acessada com sucesso.") 
else: print(f"Erro ao acessar a pasta normal_dir.") 
if os.path.exists(cancer_dir): print(f"Pasta cancer_dir acessada com sucesso.") 
else: print(f"Erro ao acessar a pasta cancer_dir.")

if os.listdir(normal_dir):print("sim, na pasta normal ha itens")
if os.listdir(cancer_dir):print("sim, na pasta cancer ha itens")


Pasta normal_dir acessada com sucesso.
Pasta cancer_dir acessada com sucesso.
sim, na pasta normal ha itens
sim, na pasta cancer ha itens


In [25]:
# Criar um DataFrame para armazenar os resultados
columns = ['Modelo', 'Acuracia', 'Sensibilidade', 'Especificidade', 'F-Score', 'AUC-ROC']
df_metrics = pd.DataFrame(columns=columns)

# Definir o caminho para salvar o melhor modelo
model_checkpoint_path = r"C:\Users\andre\Documents\atividades\estudos_opencv\processamento de imagens\\best_model.h5"

In [26]:
# Iterar sobre os modelos
def model_execution(model_name):

    # Carregamento de imagens e conversão para XYZ
    normal_images = load_images(normal_dir, 'rgb',img_extensions)
    cancer_images = load_images(cancer_dir, 'rgb',img_extensions)

    # Rótulos para imagens (0 para normal, 1 para câncer)
    normal_labels = np.zeros(normal_images.shape[0])
    cancer_labels = np.ones(cancer_images.shape[0])

    # Concatenar imagens e rótulos
    all_images = np.concatenate([normal_images, cancer_images], axis=0)
    all_labels = np.concatenate([normal_labels, cancer_labels], axis=0)

    # Dividir o conjunto de dados em treino e teste (80/20)
    X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)

    # Escolha entre 5 tipos de conversão de escala de cor aqui (por exemplo, normalização)
    X_train = X_train / 255.0
    X_test = X_test / 255.0

    # Escolher o modelo de CNN
    if model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'MobileNetV2':
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'InceptionV3':
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'EfficientNetB0':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'Xception':
        base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'VisionTransformer':
            base_model = VitB16(image_size=(224, 224), weights='imagenet', include_top=False)
    else:
        raise ValueError("Modelo de CNN não suportado.")

    # Construir o modelo
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compilar o modelo
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Configurar o retorno de chamada ModelCheckpoint
#     checkpoint = callbacks.ModelCheckpoint(model_checkpoint_path,
#                                            monitor='val_accuracy',  # Métrica a ser monitorad
#                                            mode='max',               # Salvar o modelo com a maior precisão
#                                            verbose=1)

    # Treinar o modelo
    model.fit(X_train, y_train, epochs=60, batch_size=16, validation_split=0.2)

    # Avaliar o modelo
    y_pred = model.predict(X_test)
    y_pred_binary = (y_pred > 0.5).astype(int)

    # Calcular métricas
    metrics = get_metrics(y_test, y_pred_binary)

    # Adicionar as métricas ao DataFrame
    metrics['Modelo'] = model_name
    
    # Excluir o modelo atual para liberar memória da GPU   
    del all_images, normal_images, cancer_images, X_train, X_test, y_train, y_test, y_pred, y_pred_binary
    del all_labels, normal_labels, cancer_labels
    tf.keras.backend.clear_session()
    del model, base_model
    gc.collect()
    
    return metrics


In [27]:
chosen_model = 'VGG16'
metrics = model_execution(chosen_model)
metrics = model_execution(chosen_model)
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics = pd.concat([df_metrics, temp_df], ignore_index=True)

chosen_model = 'ResNet50'
metrics = model_execution(chosen_model)
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics = pd.concat([df_metrics, temp_df], ignore_index=True)

chosen_model = 'InceptionV3'
metrics = model_execution(chosen_model)
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics = pd.concat([df_metrics, temp_df], ignore_index=True)

chosen_model = 'MobileNetV2'
metrics = model_execution(chosen_model)
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics = pd.concat([df_metrics, temp_df], ignore_index=True)

chosen_model = 'EfficientNetB0'
metrics = model_execution(chosen_model)
temp_df = pd.DataFrame([[chosen_model, metrics['accuracy'], metrics['recall'], metrics['specificity'], metrics['f1-score'], metrics['auc-roc']]], columns=columns)  
df_metrics = pd.concat([df_metrics, temp_df], ignore_index=True)

df_metrics

Epoch 1/60
8/8 [==============================] - 2s 163ms/step - loss: 0.7405 - accuracy: 0.5440 - val_loss: 0.6426 - val_accuracy: 0.4688
Epoch 2/60
8/8 [==============================] - 1s 142ms/step - loss: 0.6584 - accuracy: 0.6000 - val_loss: 0.5897 - val_accuracy: 0.7500
Epoch 3/60
8/8 [==============================] - 1s 142ms/step - loss: 0.4835 - accuracy: 0.8000 - val_loss: 0.5402 - val_accuracy: 0.7812
Epoch 4/60
8/8 [==============================] - 2s 256ms/step - loss: 0.4292 - accuracy: 0.8000 - val_loss: 0.5818 - val_accuracy: 0.7812
Epoch 5/60
8/8 [==============================] - 1s 143ms/step - loss: 0.2433 - accuracy: 0.8960 - val_loss: 0.5797 - val_accuracy: 0.8438
Epoch 6/60
8/8 [==============================] - 1s 142ms/step - loss: 0.2796 - accuracy: 0.8640 - val_loss: 0.4000 - val_accuracy: 0.8750
Epoch 7/60
8/8 [==============================] - 1s 141ms/step - loss: 0.2337 - accuracy: 0.8960 - val_loss: 0.3384 - val_accuracy: 0.8438
Epoch 8/60
8/8 [====

c:\Users\andre\anaconda3\envs\TFenv\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  


Epoch 1/60
8/8 [==============================] - 5s 206ms/step - loss: 0.8477 - accuracy: 0.5440 - val_loss: 1.1833 - val_accuracy: 0.6250
Epoch 2/60
8/8 [==============================] - 1s 111ms/step - loss: 0.3817 - accuracy: 0.8640 - val_loss: 1.2455 - val_accuracy: 0.6250
Epoch 3/60
8/8 [==============================] - 1s 110ms/step - loss: 0.1310 - accuracy: 0.9600 - val_loss: 1.2570 - val_accuracy: 0.6562
Epoch 4/60
8/8 [==============================] - 1s 111ms/step - loss: 0.0437 - accuracy: 0.9760 - val_loss: 0.7951 - val_accuracy: 0.7188
Epoch 5/60
8/8 [==============================] - 1s 111ms/step - loss: 0.0150 - accuracy: 0.9920 - val_loss: 0.8884 - val_accuracy: 0.7188
Epoch 6/60
8/8 [==============================] - 1s 113ms/step - loss: 0.0174 - accuracy: 0.9920 - val_loss: 0.6272 - val_accuracy: 0.7812
Epoch 7/60
8/8 [==============================] - 1s 113ms/step - loss: 0.0084 - accuracy: 1.0000 - val_loss: 0.6076 - val_accuracy: 0.7812
Epoch 8/60
8/8 [====

,Modelo,Acuracia,Sensibilidade,Especificidade,F-Score,AUC-ROC
0,VGG16,0.900,1.000000,0.809524,0.904762,0.904762
1,ResNet50,0.525,0.000000,1.000000,0.000000,0.500000
2,InceptionV3,0.950,0.947368,0.952381,0.947368,0.949875
3,MobileNetV2,0.950,0.947368,0.952381,0.947368,0.949875
4,EfficientNetB0,0.550,0.052632,1.000000,0.100000,0.526316
